In [1]:
import findspark
findspark.init('C:/Users/Laurens/Downloads/spark-2.3.0-bin-hadoop2.7') # fill in your own home file where spark is located

from datetime import datetime
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
import pandas as pd
import requests
from time import gmtime, strftime
from datetime import datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import date_format
from pyspark.sql.functions import lit
%matplotlib inline 

In [2]:
import Neo4J_connector as db

In [3]:
bitcoins = ['#bitcoin', '#ethereum', '#ripple','#bitcoin-cash','#eos','#litecoin', '#cardano', '#stellar', '#iota', '#tron']

In [4]:
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()

ssc = StreamingContext(sc, 60)
sqlContext = SQLContext(sc)

socket_stream = ssc.socketTextStream("127.0.0.1", 5555)

lines = socket_stream.window( 120 )

In [5]:
from collections import namedtuple
fields = ("tag", "count")
Tweet = namedtuple( 'Tweet', fields )


In [6]:
# Get bitcoin data from Coinmarketcap
def get_price(coin1):
    '''
    Call API for getting the bitcoin price.
    coin: 
    '''
    url1 = "https://api.coinmarketcap.com/v1/ticker/"
    prices1= {}
    response1=requests.get(url1)
    for i in range(10):
        prices1[response1.json()[i]["id"]] = float(response1.json()[i]["price_usd"])
    return  prices1[coin1[1:]]
                
get_coin_value = udf(get_price, FloatType())    

def get_percent_change_1h(coin2):
    url2 = "https://api.coinmarketcap.com/v1/ticker/"
    prices2= {}
    response2=requests.get(url2)
    for i in range(10):
        prices2[response2.json()[i]["id"]] = float(response2.json()[i]["percent_change_1h"])
    return  prices2[coin2[1:]]
          
get_coin_change_1h = udf(get_percent_change_1h, FloatType())  

def get_market_cap_usd(coin3):
    '''
    Call API for getting the bitcoin price.
    coin: 
    '''
    url3 = "https://api.coinmarketcap.com/v1/ticker/"
    prices3= {}
    response3=requests.get(url3)
    
    for i in range(10):
        prices3[response3.json()[i]["id"]] = float(response3.json()[i]["market_cap_usd"])
        
    return  prices3[coin3[1:]]
                

get_coin_market_cap_usd = udf(get_market_cap_usd, FloatType()) 


In [7]:
(lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
    .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
    .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word # Checks for hashtag calls
    .filter( lambda word:word[0] in bitcoins )
    .reduceByKey( lambda a, b: a + b ) # Reduces
    .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
    .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") )# Sorts Them in a DF
    .select('tag','count',
            get_coin_value('tag').alias('Price'),
            get_coin_change_1h('tag').alias('Percent_change_1h'),
            get_coin_market_cap_usd('tag').alias('Market_cap_usd'))
    .withColumn('Time',lit(str(strftime("%H:%M", time.localtime()))))
    .limit(50).registerTempTable("tweets") ) ) # Registers to a table.

In [8]:
schema= StructType([
    StructField("tag", StringType(), True),
    StructField("count", IntegerType(), True),
    StructField("Price", FloatType(), True),
    StructField("Percent_change_1h", FloatType(), True),
    StructField("Market_cap_usd", FloatType(), True),
    StructField("Time", StringType(), True)  
    ])


TotalDateFrame = sqlContext.createDataFrame(sc.emptyRDD(), schema)

In [9]:
TotalDateFrame.printSchema()

root
 |-- tag: string (nullable = true)
 |-- count: integer (nullable = true)
 |-- Price: float (nullable = true)
 |-- Percent_change_1h: float (nullable = true)
 |-- Market_cap_usd: float (nullable = true)
 |-- Time: string (nullable = true)



# Run the  TweetRead.py from terminal

In [10]:
ssc.start()

In [ ]:
count = 0
flag = 1

query_time = []
time_stamps = []
while count < 150:
    
    time.sleep(120)
    
    start_time = time.time()
    
    top_10_tweets = sqlContext.sql( 'Select * from tweets')
    print(top_10_tweets.count())

    temp = top_10_tweets.toPandas()
    
    for i in range(0, top_10_tweets.count()):
        db.create_tweetcount(temp['tag'][i][1:],
                             str(temp['count'][i]), 
                             str(temp['Price'][i]), 
                             str(temp['Time'][i]))
#             else: # make only bitcoin object for Neo4J
#                 db.create_new_datapoint_crypto()

    print(top_10_tweets.show())
    print('--')
   
    #TotalDateFrame= TotalDateFrame.union(top_10_tweets)
   
    if flag ==1:
        #TotalDateFrame = top_10_tweets.rdd
        
        total = top_10_tweets.toPandas()
        flag =0
    else: 
        
        wind = top_10_tweets.toPandas()
        total = pd.concat([total,wind])
        
        #TotalDateFrame= TotalDateFrame.union(top_10_tweets.rdd)

    print("--- %s seconds ---" % (time.time() - start_time))   
    
    query_time.append(str(time.time() - start_time))
    time_stamps.append(str(strftime("%H:%M", time.localtime())))
    
    print('----------') 
    count = count + 1  

1


In [13]:
total.shape

(355, 6)

In [15]:
top_10_tweets = sqlContext.sql( 'Select * from tweets ')
TotalDateFrame= TotalDateFrame.union(top_10_tweets)

In [16]:
top_10_tweets.show()

+---------+-----+--------+-----------------+--------------+-----+
|      tag|count|   Price|Percent_change_1h|Market_cap_usd| Time|
+---------+-----+--------+-----------------+--------------+-----+
| #bitcoin|   17| 7438.97|             0.01| 1.27028224E11|08:56|
|#ethereum|   13| 587.967|             0.36|  5.8725065E10|08:56|
|  #ripple|    1|0.656052|             0.43| 2.57444823E10|08:56|
|     #eos|    1| 13.1961|             0.36| 1.18256783E10|08:56|
| #cardano|    1|0.208764|             0.54|   5.4126387E9|08:56|
| #stellar|    1|0.285228|              0.2|   5.2995768E9|08:56|
|    #iota|    1| 1.68682|             0.23|   4.6885673E9|08:56|
+---------+-----+--------+-----------------+--------------+-----+



In [17]:
TotalDateFrame.show()

+---------+-----+--------+-----------------+--------------+-----+
|      tag|count|   Price|Percent_change_1h|Market_cap_usd| Time|
+---------+-----+--------+-----------------+--------------+-----+
| #bitcoin|   17| 7438.97|             0.01| 1.27028224E11|08:56|
|#ethereum|   13| 587.967|             0.36|  5.8725065E10|08:56|
|  #ripple|    1|0.656052|             0.43| 2.57444823E10|08:56|
|     #eos|    1| 13.1961|             0.36| 1.18256783E10|08:56|
| #cardano|    1|0.208764|             0.54|   5.4126387E9|08:56|
| #stellar|    1|0.285228|              0.2|   5.2995768E9|08:56|
|    #iota|    1| 1.68682|             0.23|   4.6885673E9|08:56|
+---------+-----+--------+-----------------+--------------+-----+



In [18]:
import time
start_time = time.time()


Total=TotalDateFrame.toPandas()



print("--- %s seconds ---" % (time.time() - start_time))

--- 107.00478172302246 seconds ---


In [19]:
Total.head()

,tag,count,Price,Percent_change_1h,Market_cap_usd,Time
0,#bitcoin,17,7438.970215,0.01,1.270282e+11,08:56
1,#ethereum,13,587.966980,0.36,5.872506e+10,08:56
2,#ripple,1,0.656052,0.43,2.574448e+10,08:56
3,#eos,1,13.196100,0.36,1.182568e+10,08:56
4,#cardano,1,0.208764,0.54,5.412639e+09,08:56


In [20]:
from matplotlib import dates

total1['Time'] = pd.to_datetime(total['Time'] )

plt.figure( figsize = ( 10, 8 ) )

ax = total1[total1['tag']=='#bitcoin'].plot(x='Time', y='count',grid = True,title = 'Bitcoin')
# ax.xaxis.set_minor_formatter(dates.DateFormatter("%H:%M"))
total1[total1['tag']=='#bitcoin'].plot(x='Time', y='Price',grid = True)
plt.show()

NameError: name 'total1' is not defined

# Playground

In [ ]:
file_name= 'skata.csv'
Total.to_csv(file_name,index=False)

In [ ]:
TotalDateFrame.show()

In [ ]:
# Method 2: Use SQL
x=TotalDateFrame.registerTempTable("FinalTweetsCoin")

In [ ]:
x=sqlContext.sql( 'Select MAX(price) as maxval from TweetsCoin').collect()[0].asDict()['maxval']

x.show()

In [ ]:
TotalDateFrame.sort('Time', ascending=True).show()

In [ ]:
Total=TotalDateFrame.toPandas()
Total['Time'] = pd.to_datetime(Total['Time'] )

In [15]:
total1 = total

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

In [ ]:
ssc.stop()

In [ ]:
TotalDateFrame.write.csv('/bitcoins.csv')